# PriMAT: Multi-animal tracking

In this notebook, we want to demonstrate how to train a tracking model from a few hundred frames. We will train a model that is able to track lemurs and feeding boxes with labelled images which can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Training Images / LemurBoxSep22 ).

In [1]:
!nvidia-smi

Thu Dec 11 10:19:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:

# =========================
# Colab: installation via pip (versions épinglées)
# =========================
import sys, subprocess

def pip_install(*pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-U"]
    cmd += list(pkgs)
    print(">>>", " ".join(cmd))
    subprocess.check_call(cmd)

# (Optionnel) accélérer les résolutions
pip_install("pip==23.1")

# Paquets "conda-like" traduits en pip
# NOTE: pour GPU, les wheels très anciens (torch 1.7 + cu102) peuvent être introuvables sur Colab.
#       Si erreur CUDA, reste en CPU (torch sans suffixe CUDA) ou mets à jour torch/torchvision.
pip_install(
    "numpy==1.20.0",
    "Cython==0.29.*",
    "matplotlib==3.5.*",
    "pandas==1.4.*",
    "Pillow==9.4.*",
    "pycocotools==2.0.*",
    "pymediainfo",
    "seaborn==0.12.*",
    "torchmetrics==0.10.*",
    "attrs==21.4.0",
    "entrypoints==0.3",
    "iprogress==0.4",
    "joblib==1.3.1",
    "pytz==2021.3",
    "PyYAML==6.0",
    "scikit-learn==1.3.0",
    "threadpoolctl==3.1.0",
)

# Torch / Torchvision (CPU-only recommandé avec ces versions)
# Si tu veux absolument CUDA 10.2 avec torch==1.7, il faudrait des wheels cu102 historiques,
# souvent indisponibles sur Colab. Le CPU-only évite ces incompatibilités.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# =========================
# Vérifications rapides
# =========================
print("\n=== Vérifications ===")
def check_imports():
    import numpy as np
    import torch, torchvision
    import pandas as pd, matplotlib, PIL
    import pycocotools, seaborn, pymediainfo, torchmetrics
    import sklearn
    print("Python :", sys.version)
    print("NumPy :", np.__version__)
    print("Torch :", torch.__version__, "| Torchvision :", torchvision.__version__)
    print("Pandas:", pd.__version__, "| Matplotlib:", matplotlib.__version__)
    print("scikit-learn:", sklearn.__version__)
    print("Pillow:", PIL.__version__, "| seaborn:", seaborn.__version__)
    print("pycocotools OK:", hasattr(pycocotools, "__version__"))
    print("pymediainfo OK:", hasattr(pymediainfo, "__version__"))
    # CUDA dispo ? (probablement False avec CPU-only)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))

try:
    import pymediainfo
    check_imports()
except Exception as e:
    print("Erreur de vérification:", e)


>>> /usr/bin/python3 -m pip install -U pip==23.1
>>> /usr/bin/python3 -m pip install -U numpy==1.20.0 Cython==0.29.* matplotlib==3.5.* pandas==1.4.* Pillow==9.4.* pycocotools==2.0.* pymediainfo==6.1.* seaborn==0.12.* torchmetrics==0.10.* attrs==21.4.0 entrypoints==0.3 iprogress==0.4 joblib==1.3.1 pytz==2021.3 PyYAML==6.0 scikit-learn==1.3.0 threadpoolctl==3.1.0
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.3 MB/s eta 0:00:00

=== Vérifications ===
Erreur de vérification: No module named 'pymediainfo'


In [4]:
!sudo apt-get install -y mediainfo


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmediainfo0v5 libmms0 libtinyxml2-9 libzen0v5
Suggested packages:
  mediainfo-gui
The following NEW packages will be installed:
  libmediainfo0v5 libmms0 libtinyxml2-9 libzen0v5 mediainfo
0 upgraded, 5 newly installed, 0 to remove and 75 not upgraded.
Need to get 2,440 kB of archives.
After this operation, 7,566 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmms0 amd64 0.6.4-3 [27.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtinyxml2-9 amd64 9.0.0+dfsg-3 [32.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libzen0v5 amd64 0.4.39-1 [97.5 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmediainfo0v5 amd64 21.09+dfsg-4 [2,255 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mediainfo amd64 22.03-1 [28.5 kB]


In [5]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install yacs==0.1.8
!pip install opencv-python
!pip install progress==1.6
!pip install scikit-learn==1.2.2

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9612 sha256=fc94caba2456860cafb28c0008836ba9a3889c6a0e8a81d8aab3758a9e0af39d
  Stored in directory: /root/.cache/pip/wheels/bb/01/5a/c916509df9b12c6465864251dbe826def8e31a16fa7da54f08
Successfully built progress
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 39.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [6]:
!git clone https://github.com/ecker-lab/PriMAT-tracking.git
%cd PriMAT-tracking/
!mkdir data
!mkdir models
!mkdir exp
!mkdir videos

Cloning into 'PriMAT-tracking'...
remote: Enumerating objects: 8971, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8971 (delta 0), reused 0 (delta 0), pack-reused 8968 (from 2)
Receiving objects: 100% (8971/8971), 144.93 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (1789/1789), done.
Updating files: 100% (8972/8972), done.
/content/PriMAT-tracking


## Training

### Extract data

You have to upload the folder with the data to the colab space on the left. I uploaded it as a tar file and extracted it into the folder "data". The only important thing is that after this step you have training material in the folder data.

In [ ]:
%cd data
!tar -xvf ../../LemurBoxSep22.tar >/dev/null!tar
%cd ..

### Run training script

In the following cell you start the training.

1.   Élément de liste
2.   Élément de liste

Before this you have to make sure a few things:
- The data_cfg file has to point to the correct location of your data. Open it (it is in PriMAT-tracking/src/lib/cfg/lemur-box.json) and adapt the root (for me it is /content/PriMAT-tracking/data/ in colab).
- Give your experiment a name (--exp_id) so that you can find the model afterwards in exp/mot/exp_id/.

In [ ]:
%%shell
cd src
python train.py mot --exp_id colab_test\
                    --load_model ''\
                    --num_epochs 10\
                    --lr_step 5\
                    --lr '1e-4'\
                    --data_cfg '../src/lib/cfg/lemur_box.json'\
                    --store_opt\
                    --arch hrnet_32\
                    --gpus 0\
                    --batch_size 2\
                    --seed 13\
                    --reid_cls_names lemur,box\
                    --val_intervals 10\
                    --save_all
cd ..

Your models will be saved in exp/mot/exp_id and end with .pth.

## Inference

### Apply to videos

If you want to have video output, you will need to activate ffmpeg.

In [ ]:
!apt-get update && apt-get install -y ffmpeg

- If you want the model you just trained, you can directly change the path to ../exp/mot/exp_id/model_last.pth (or any other model you want). Alternatively, we can use the pretrained lemur model from [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> Models).
- You can upload your own videos or a validation video (e.g. Eval8.mp4) into data/Videos. Videos can be downloaded [here](https://owncloud.gwdg.de/index.php/s/Mq4m9k1B74cN6ys) (-> ValidationVideos/lemur_videos_eval/Videos/).
- If you set output_format to video, a video will be saved to output_root/output_name. If you set it to text, only the tracking output as a .txt file will be saved.

In [ ]:
!pip install motmetrics==1.2.0
!pip install lap==0.4.0
!pip install cython_bbox==0.1.3

In [ ]:
%%shell

cd src

python demo.py mot  --load_tracking_model ../models/lemur_tracking.pth\
                    --conf_thres 0.02\
                    --det_thres 0.5\
                    --new_overlap_thres 0.8\
                    --sim_thres 0.8\
                    --input_video ../data/Videos/Eval8.mp4\
                    --output_root ../videos/test/\
                    --output_name test_video\
                    --store_opt\
                    --line_thickness 2\
                    --debug_info\
                    --arch hrnet_32\
                    --output_format video\
                    --reid_cls_names "lemur,box"\
                    --proportion_iou 0.2\
                    --double_kalman


cd ..